In [36]:
# Dependencies
import pandas as pd

# cases_death Table CleanUp

In [38]:
# Load & read in maskusage file
data_load = "../RawData/county_cases.csv"
cases_data = pd.read_csv(data_load,  encoding = 'unicode_escape')
cases_data

,date,county,state,County_Sate,fips,cases,deaths,confirmed_cases,confirmed_deaths,probable_cases,probable_deaths
0,10/14/2020,Autauga,Alabama,"Autauga County, Alabama",1001.0,1928,28.0,1715.0,27.0,213.0,1.0
1,10/14/2020,Baldwin,Alabama,"Baldwin County, Alabama",1003.0,6270,66.0,5400.0,62.0,870.0,4.0
2,10/14/2020,Barbour,Alabama,"Barbour County, Alabama",1005.0,950,9.0,693.0,9.0,257.0,0.0
3,10/14/2020,Bibb,Alabama,"Bibb County, Alabama",1007.0,744,13.0,697.0,9.0,47.0,4.0
4,10/14/2020,Blount,Alabama,"Blount County, Alabama",1009.0,1750,23.0,1355.0,23.0,395.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3239,10/14/2020,Unknown,Tennessee,"Unknown County, Tennessee",NaN,129,0.0,41.0,NaN,88.0,NaN
3240,10/14/2020,Unknown,Utah,"Unknown County, Utah",NaN,211,1.0,25.0,1.0,186.0,0.0
3241,10/14/2020,Unknown,Vermont,"Unknown County, Vermont",NaN,8,0.0,8.0,0.0,NaN,NaN
3242,10/14/2020,Unknown,Washington,"Unknown County, Washington",NaN,2,2.0,0.0,0.0,2.0,2.0


In [40]:
#select desired columns
cases_data = pd.DataFrame(cases_data,columns= ['fips','cases','deaths'])
cases_data


,fips,cases,deaths
0,1001.0,1928,28.0
1,1003.0,6270,66.0
2,1005.0,950,9.0
3,1007.0,744,13.0
4,1009.0,1750,23.0
...,...,...,...
3239,NaN,129,0.0
3240,NaN,211,1.0
3241,NaN,8,0.0
3242,NaN,2,2.0


In [42]:
#Remove blank fips from DF
cases_data = cases_data.dropna(subset = ['fips'])
cases_data

,fips,cases,deaths
0,1001.0,1928,28.0
1,1003.0,6270,66.0
2,1005.0,950,9.0
3,1007.0,744,13.0
4,1009.0,1750,23.0
...,...,...,...
3209,72151.0,426,NaN
3210,72153.0,461,NaN
3211,78010.0,491,6.0
3212,78020.0,65,1.0


In [44]:
#Remove Fips that do not have population data 
cases_data = cases_data.loc[cases_data['fips']<57000]
cases_data

,fips,cases,deaths
0,1001.0,1928,28.0
1,1003.0,6270,66.0
2,1005.0,950,9.0
3,1007.0,744,13.0
4,1009.0,1750,23.0
...,...,...,...
3126,56037.0,401,2.0
3127,56039.0,679,1.0
3128,56041.0,384,2.0
3129,56043.0,126,7.0


In [46]:
#convert fips column to a string 
cases_data['fips']=cases_data['fips'].astype(str)
cases_data.head()

,fips,cases,deaths
0,1001.0,1928,28.0
1,1003.0,6270,66.0
2,1005.0,950,9.0
3,1007.0,744,13.0
4,1009.0,1750,23.0


In [48]:
cases_data.dtypes

fips       object
cases       int64
deaths    float64
dtype: object

In [50]:
#Remove extra decimnals from fips string column 
cases_data['fips']= cases_data['fips'].str[:-2]
cases_data

,fips,cases,deaths
0,1001,1928,28.0
1,1003,6270,66.0
2,1005,950,9.0
3,1007,744,13.0
4,1009,1750,23.0
...,...,...,...
3126,56037,401,2.0
3127,56039,679,1.0
3128,56041,384,2.0
3129,56043,126,7.0


In [52]:
#Reanme and create index on DF
cases_data = cases_data.rename(columns= {'fips': 'county_fips'}).set_index('county_fips')


In [54]:
# load and read county_state for state_fips
data2_load = "county_state.csv"
county_state_data = pd.read_csv(data2_load, header=0,  encoding = 'unicode_escape')
county_state_data.head()

,county_fips,state_fips
0,1001,1000
1,1003,1000
2,1005,1000
3,1007,1000
4,1009,1000


In [56]:
#convert county fips from int to str
county_state_data['county_fips']= county_state_data['county_fips'].astype(str)
print(county_state_data.dtypes)
print(cases_data.dtypes)

county_fips    object
state_fips      int64
dtype: object
cases       int64
deaths    float64
dtype: object


In [58]:
# Merge state fips into cases_data DF 
cases_data= pd.merge(cases_data, county_state_data, how= 'left', on= 'county_fips')
# Reorder columns in same order as ERD and set index
columns_order = ['county_fips', 'state_fips', 'cases', 'deaths']
cases_data = cases_data.reindex(columns=columns_order).set_index('county_fips')
cases_data

,state_fips,cases,deaths
county_fips,,,
1001,1000,1928,28.0
1003,1000,6270,66.0
1005,1000,950,9.0
1007,1000,744,13.0
1009,1000,1750,23.0
...,...,...,...
56037,56000,401,2.0
56039,56000,679,1.0
56041,56000,384,2.0


In [59]:
#save DF to csv 
cases_data.to_csv('cases_death.csv')

# mask_usage Table CleanUp

In [60]:
# Load & read in geocodes file
data_load = "../RawData/mask-use-by-county.csv"
mask_data = pd.read_csv(data_load, header=0,  encoding = 'unicode_escape')
mask_data.head()

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,1001,5.3%,7.4%,13.4%,29.5%,44.4%
1,1003,8.3%,5.9%,9.8%,32.3%,43.6%
2,1005,6.7%,12.1%,12.0%,20.1%,49.1%
3,1007,2.0%,3.4%,9.6%,27.8%,57.2%
4,1009,5.3%,11.4%,18.0%,19.4%,45.9%


In [61]:
# Add state_fips from county_state into dataframe
mask_data['state_fips'] = county_state_data['state_fips']

# Reanme columns
mask_usage = mask_data.rename(columns={'COUNTYFP': 'county_fips', 'NEVER':'never', 'RARELY':'rarely', 'SOMETIMES':'sometimes', 'FREQUENTLY':'frequently', 'ALWAYS': 'always'})

# Reorder columns in same order as ERD and set index
columns_order = ['county_fips', 'state_fips', 'never', 'rarely', 'sometimes', 'frequently', 'always']
mask_usage = mask_usage.reindex(columns=columns_order).set_index('county_fips')

# Save file to clean data
mask_usage.to_csv('mask_usage.csv')

# population tabel CleanUp

In [62]:
# Load & read in geocodes file
data_load = "../RawData/county_census.csv"
census_data = pd.read_csv(data_load, header=3, encoding = 'unicode_escape')
census_data.head()

,Unnamed: 0,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,"308,745,538","308,758,105","309,321,666","311,556,874","313,830,990","315,993,715","318,301,008","320,635,163","322,941,311","324,985,539","326,687,501","328,239,523"
1,".Autauga County, Alabama","54,571","54,597","54,773","55,227","54,954","54,727","54,893","54,864","55,243","55,390","55,533","55,869"
2,".Baldwin County, Alabama","182,265","182,265","183,112","186,558","190,145","194,885","199,183","202,939","207,601","212,521","217,855","223,234"
3,".Barbour County, Alabama","27,457","27,455","27,327","27,341","27,169","26,937","26,755","26,283","25,806","25,157","24,872","24,686"
4,".Bibb County, Alabama","22,915","22,915","22,870","22,745","22,667","22,521","22,553","22,566","22,586","22,550","22,367","22,394"


In [63]:
# Select wanted columns/drop united states totals/drop notes in file 
population = pd.DataFrame(census_data, columns=['2019'])
population = population.drop(population.index[3143:3149])
population = population.drop(0)
population.reset_index(drop=True, inplace=True)

In [64]:
# insert county_fips and state_fips
population['state_fips'] = county_state_data['state_fips']
population['county_fips'] = county_state_data['county_fips']
population.head()

,2019,state_fips,county_fips
0,"55,869",1000,1001
1,"223,234",1000,1003
2,"24,686",1000,1005
3,"22,394",1000,1007
4,"57,826",1000,1009


In [65]:
# Reorder columns in same order as ERD and set index
columns_order = ['county_fips', 'state_fips', '2019']
population = population.reindex(columns=columns_order).set_index('county_fips')

# Rename population column
population = population.rename(columns={'2019': '2019_population'})

# save population to csv
population.to_csv('population.csv')
population.head()

,state_fips,2019_population
county_fips,,
1001,1000,"55,869"
1003,1000,"223,234"
1005,1000,"24,686"
1007,1000,"22,394"
1009,1000,"57,826"


# Dataframe to postgres

In [66]:
# import dependencies
from sqlalchemy import create_engine
import psycopg2
import pandas as pd
from passwords import password

## Cases_death table

In [67]:
# Create an engine instance
alchemyEngine   = create_engine('postgresql+psycopg2://postgres:' + password + '@cwru-bootcamp6.cbgqvzvry5u3.us-east-2.rds.amazonaws.com/postgres', pool_recycle=3600)

# Connect to PostgreSQL server
postgreSQLConnection = alchemyEngine.connect()
postgreSQLTable = "cases_death"

try:

    frame = cases_data.to_sql(postgreSQLTable, postgreSQLConnection, if_exists='fail')

except ValueError as vx:

    print(vx)

except Exception as ex:  

    print(ex)

else:

    print("PostgreSQL Table %s has been created successfully."%postgreSQLTable)

finally:

    postgreSQLConnection.close()

Table 'cases_death' already exists.


## Mask_usage table

In [68]:
# Create an engine instance
alchemyEngine   = create_engine('postgresql+psycopg2://postgres:' + password + '@cwru-bootcamp6.cbgqvzvry5u3.us-east-2.rds.amazonaws.com/postgres', pool_recycle=3600)

# Connect to PostgreSQL server
postgreSQLConnection = alchemyEngine.connect()
postgreSQLTable = "mask_usage"

try:

    frame = mask_usage.to_sql(postgreSQLTable, postgreSQLConnection, if_exists='fail')

except ValueError as vx:

    print(vx)

except Exception as ex:  

    print(ex)

else:

    print("PostgreSQL Table %s has been created successfully."%postgreSQLTable)

finally:

    postgreSQLConnection.close()

Table 'mask_usage' already exists.


## Population table

In [69]:
# Create an engine instance
alchemyEngine   = create_engine('postgresql+psycopg2://postgres:' + password + '@cwru-bootcamp6.cbgqvzvry5u3.us-east-2.rds.amazonaws.com/postgres', pool_recycle=3600)

# Connect to PostgreSQL server
postgreSQLConnection = alchemyEngine.connect()
postgreSQLTable = "population"

try:

    frame = population.to_sql(postgreSQLTable, postgreSQLConnection, if_exists='fail')

except ValueError as vx:

    print(vx)

except Exception as ex:  

    print(ex)

else:

    print("PostgreSQL Table %s has been created successfully."%postgreSQLTable)

finally:

    postgreSQLConnection.close()

Table 'population' already exists.
